In [ ]:
import unittest
from unittest import mock
from unittest.mock import Mock, patch, AsyncMock
from http import HTTPStatus
import os

from your_package.utils.conversation_utils import ConversationManagementUtils


class TestConversationManagementUtils(unittest.TestCase):
    def setUp(self):
        self.app_id = "test_app"
        self.request_handler = Mock()
        self.utils = ConversationManagementUtils(
            app_id=self.app_id,
            request_handler=self.request_handler,
            semaphore_limit=10
        )
    
    def test_create_headers(self):
        headers = self.utils._create_headers("test_sso")
        self.assertEqual(headers["Cookie"], "SSO=test_sso")
        self.assertEqual(headers["app-id"], self.app_id)
    
    def test_get_all_conversations_success(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.return_value = [
            {"conversationId": "conv1"},
            {"conversationId": "conv2"}
        ]
        self.request_handler.get.return_value = mock_response
        
        result = self.utils.get_all_conversations("test_sso")
        self.assertEqual(result, ["conv1", "conv2"])
        
        self.request_handler.get.assert_called_once()
        args, kwargs = self.request_handler.get.call_args
        self.assertTrue(kwargs["path"].endswith("/conversations"))
    
    def test_get_all_conversations_failure(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.BAD_REQUEST
        mock_response.text = "Error text"
        self.request_handler.get.return_value = mock_response
        
        result = self.utils.get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    def test_get_all_conversations_exception(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.side_effect = Exception("Test error")
        self.request_handler.get.return_value = mock_response
        
        result = self.utils.get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    def test_get_all_conversations_none_response(self):
        self.request_handler.get.return_value = None
        
        result = self.utils.get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    async def test_async_get_all_conversations_success(self):
        mock_response = Mock()
        mock_response.status = HTTPStatus.OK
        mock_response.json = AsyncMock(return_value=[
            {"conversationId": "conv1"},
            {"conversationId": "conv2"}
        ])
        self.request_handler.async_get = AsyncMock(return_value=mock_response)
        
        result = await self.utils.async_get_all_conversations("test_sso")
        self.assertEqual(result, ["conv1", "conv2"])
    
    async def test_async_get_all_conversations_failure(self):
        mock_response = Mock()
        mock_response.status = HTTPStatus.BAD_REQUEST
        mock_response.text = AsyncMock(return_value="Error text")
        self.request_handler.async_get = AsyncMock(return_value=mock_response)
        
        result = await self.utils.async_get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    async def test_async_get_all_conversations_exception(self):
        mock_response = Mock()
        mock_response.status = HTTPStatus.OK
        mock_response.json = AsyncMock(side_effect=Exception("Test error"))
        self.request_handler.async_get = AsyncMock(return_value=mock_response)
        
        result = await self.utils.async_get_all_conversations("test_sso")
        self.assertEqual(result, [])
    
    def test_get_conversation_id_success(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.return_value = {
            "conversationId": "conv123",
            "data": "test_data"
        }
        self.request_handler.post.return_value = mock_response
        
        conv_id, data = self.utils.get_conversation_id("test_sso", {"pref": "value"})
        self.assertEqual(conv_id, "conv123")
        self.assertEqual(data["data"], "test_data")
        
        args, kwargs = self.request_handler.post.call_args
        self.assertEqual(kwargs["payload"]["preferences"], {"pref": "value"})
    
    def test_get_conversation_id_failure(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.BAD_REQUEST
        mock_response.text = "Error text"
        self.request_handler.post.return_value = mock_response
        
        conv_id, data = self.utils.get_conversation_id("test_sso")
        self.assertIsNone(conv_id)
        self.assertEqual(data, {})
    
    def test_get_conversation_id_exception(self):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.side_effect = Exception("Test error")
        self.request_handler.post.return_value = mock_response
        
        conv_id, data = self.utils.get_conversation_id("test_sso")
        self.assertIsNone(conv_id)
        self.assertEqual(data, {})
    
    @patch('builtins.open', new_callable=mock.mock_open, read_data=b'test file content')
    def test_upload_success(self, mock_open):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.OK
        mock_response.json.return_value = {"document_id": "doc123"}
        self.request_handler.post.return_value = mock_response
        
        result = self.utils.upload("test_sso", "test/path.pdf")
        self.assertEqual(result, "doc123")
        
        mock_open.assert_called_once_with("test/path.pdf", "rb")
        args, kwargs = self.request_handler.post.call_args
        self.assertIn("files", kwargs)
    
    def test_upload_unsupported_file_type(self):
        result = self.utils.upload("test_sso", "test/path.unsupported")
        self.assertIsNone(result)
        self.request_handler.post.assert_not_called()
    
    @patch('builtins.open', new_callable=mock.mock_open, read_data=b'test file content')
    def test_upload_api_error(self, mock_open):
        mock_response = Mock()
        mock_response.status_code = HTTPStatus.BAD_REQUEST
        mock_response.text = "Error text"
        self.request_handler.post.return_value = mock_response
        
        result = self.utils.upload("test_sso", "test/path.pdf")
        self.assertIsNone(result)
    
    @patch('builtins.open', side_effect=IOError("File not found"))
    def test_upload_file_error(self, mock_open):
        result = self.utils.upload("test_sso", "nonexistent/path.pdf")
        self.assertIsNone(result)
        self.request_handler.post.assert_not_called()


class AsyncMock(Mock):
    async def __call__(self, *args, **kwargs):
        return super(AsyncMock, self).__call__(*args, **kwargs)


if __name__ == "__main__":
    unittest.main()

ModuleNotFoundError: No module named 'your_package'

In [ ]:
import unittest
from unittest.mock import Mock, patch, MagicMock, AsyncMock
import json
from typing import Dict, List, Any, Optional

from your_package.provider.cms import ConversationManagementService


class TestConversationManagementService(unittest.TestCase):
    def setUp(self):
        self.app_id = "test_app"
        self.env = "test_env"
        
        self.mock_request_handler = Mock()
        self.mock_utils = Mock()
        self.mock_inference_service = Mock()
        
        with patch('your_package.provider.cms.RequestHandler', return_value=self.mock_request_handler), \
             patch.object(ConversationManagementService, '_create_utils', return_value=self.mock_utils), \
             patch.object(ConversationManagementService, '_create_inference_service', return_value=self.mock_inference_service):
            
            self.service = ConversationManagementService(
                app_id=self.app_id,
                env=self.env
            )
    
    def test_initialization(self):
        self.assertEqual(self.service.app_id, self.app_id)
        self.assertEqual(self.service.utils, self.mock_utils)
        self.assertEqual(self.service.inference_service, self.mock_inference_service)
        self.assertEqual(self.service.tools, [])
    
    def test_initialization_with_tools(self):
        def test_func(): pass
        
        with patch('your_package.provider.cms.RequestHandler'), \
             patch.object(ConversationManagementService, '_create_utils'), \
             patch.object(ConversationManagementService, '_create_inference_service'):
            
            service = ConversationManagementService(
                app_id=self.app_id,
                env=self.env,
                tools=[test_func]
            )
            
            self.assertEqual(len(service.tools), 1)
            self.assertIn(test_func.__name__, service.tool_callables)
    
    def test_create_request_handler(self):
        with patch('your_package.provider.cms.RequestHandler') as mock_handler:
            self.service._create_request_handler()
            mock_handler.assert_called_once()
    
    def test_execute_tool_success(self):
        def test_func(arg1, arg2):
            return arg1 + arg2
            
        self.service.tool_callables = {"test_func": test_func}
        
        result = self.service._execute_tool("test_func", {"arg1": 1, "arg2": 2})
        self.assertEqual(result, 3)
    
    def test_execute_tool_not_found(self):
        result = self.service._execute_tool("nonexistent_func", {})
        self.assertIn("error", result)
    
    def test_execute_tool_exception(self):
        def failing_func():
            raise ValueError("Test error")
            
        self.service.tool_callables = {"failing_func": failing_func}
        
        result = self.service._execute_tool("failing_func", {})
        self.assertIn("error", result)
    
    def test_execute_function_calls(self):
        def func1(arg):
            return f"Result: {arg}"
            
        self.service.tool_callables = {"func1": func1}
        
        function_calls = [
            {"name": "func1", "args": {"arg": "test"}}
        ]
        
        results = self.service._execute_function_calls(function_calls)
        self.assertEqual(len(results), 1)
        self.assertEqual(results[0]["result"], "Result: test")
    
    def test_execute_function_calls_string_args(self):
        def func1(arg):
            return f"Result: {arg}"
            
        self.service.tool_callables = {"func1": func1}
        
        function_calls = [
            {"name": "func1", "arguments": '{"arg": "test"}'}
        ]
        
        results = self.service._execute_function_calls(function_calls)
        self.assertEqual(len(results), 1)
        self.assertEqual(results[0]["result"], "Result: test")
    
    def test_get_all_conversations(self):
        expected = ["conv1", "conv2"]
        self.mock_utils.get_all_conversations.return_value = expected
        
        result = self.service.get_all_conversations("test_sso")
        self.assertEqual(result, expected)
        self.mock_utils.get_all_conversations.assert_called_with("test_sso")
    
    async def test_async_get_all_conversations(self):
        expected = ["conv1", "conv2"]
        self.mock_utils.async_get_all_conversations = AsyncMock(return_value=expected)
        
        result = await self.service.async_get_all_conversations("test_sso")
        self.assertEqual(result, expected)
    
    def test_get_conversation(self):
        expected = ("conv123", {"data": "test"})
        self.mock_utils.get_conversation_id.return_value = expected
        
        result = self.service.get_conversation("test_sso")
        self.assertEqual(result, expected)
    
    @patch('your_package.utils.gemini_utils.GeminiUtils.extract_function_calls')
    def test_inference_with_tools_no_calls(self, mock_extract):
        mock_extract.return_value = []
        
        self.service.tools = [lambda: None]  # Add a dummy tool
        self.mock_inference_service.cms_inference.return_value = "test_response"
        
        result = self.service.inference_with_tools(
            question="test_question",
            sso="test_sso"
        )
        
        self.assertEqual(result, "test_response")
        self.mock_inference_service.cms_inference.assert_called_once()
    
    @patch('your_package.utils.gemini_utils.GeminiUtils.extract_function_calls')
    def test_inference_with_tools_with_calls(self, mock_extract):
        def test_func(param):
            return f"Executed: {param}"
            
        self.service.tools = [test_func]
        self.service.tool_callables = {"test_func": test_func}
        
        mock_extract.return_value = [
            {"name": "test_func", "args": {"param": "test_value"}}
        ]
        
        self.mock_inference_service.cms_inference.side_effect = [
            "first_response",
            "final_response"
        ]
        
        result = self.service.inference_with_tools(
            question="test_question",
            sso="test_sso"
        )
        
        self.assertEqual(result, "final_response")
        self.assertEqual(self.mock_inference_service.cms_inference.call_count, 2)
        
        # Check function results were passed to second call
        _, kwargs = self.mock_inference_service.cms_inference.call_args_list[1]
        self.assertIn("function_results", kwargs)
        self.assertEqual(len(kwargs["function_results"]), 1)
        self.assertEqual(
            kwargs["function_results"][0]["result"], 
            "Executed: test_value"
        )
    
    @patch('your_package.utils.gemini_utils.GeminiUtils.extract_function_calls')
    async def test_async_inference_with_tools(self, mock_extract):
        def test_func(param):
            return f"Executed: {param}"
            
        self.service.tools = [test_func]
        self.service.tool_callables = {"test_func": test_func}
        
        mock_extract.return_value = [
            {"name": "test_func", "args": {"param": "test_value"}}
        ]
        
        self.mock_inference_service.async_cms_inference = AsyncMock(side_effect=[
            "first_response",
            "final_response"
        ])
        
        result = await self.service.async_inference_with_tools(
            question="test_question",
            sso="test_sso"
        )
        
        self.assertEqual(result, "final_response")
        self.assertEqual(self.mock_inference_service.async_cms_inference.call_count, 2)
    
    def test_inference(self):
        self.mock_inference_service.cms_inference.return_value = "test_response"
        
        result = self.service.inference(
            question="test_question",
            sso="test_sso"
        )
        
        self.assertEqual(result, "test_response")
        self.mock_inference_service.cms_inference.assert_called_once()
    
    async def test_async_inference(self):
        self.mock_inference_service.async_cms_inference = AsyncMock(
            return_value="test_response"
        )
        
        result = await self.service.async_inference(
            question="test_question",
            sso="test_sso"
        )
        
        self.assertEqual(result, "test_response")
    
    def test_stream_inference(self):
        self.mock_utils.get_conversation_id.return_value = ("conv123", {})
        self.mock_inference_service.stream_cms_inference.return_value = "test_response"
        
        result = self.service.stream_inference(
            question="test_question",
            sso="test_sso"
        )
        
        self.assertEqual(result, "test_response")
    
    def test_stream_inference_no_conversation(self):
        self.mock_utils.get_conversation_id.return_value = (None, {})
        
        result = self.service.stream_inference(
            question="test_question",
            sso="test_sso"
        )
        
        self.assertIsNone(result)
        self.mock_inference_service.stream_cms_inference.assert_not_called()
    
    async def test_async_stream_inference(self):
        self.mock_utils.get_conversation_id.return_value = ("conv123", {})
        self.mock_inference_service.async_stream_cms_inference = AsyncMock(
            return_value="test_response"
        )
        
        result = await self.service.async_stream_inference(
            question="test_question",
            sso="test_sso"
        )
        
        self.assertEqual(result, "test_response")
    
    def test_upload_document(self):
        self.mock_utils.upload.return_value = "doc123"
        
        result = self.service.upload_document(
            sso="test_sso",
            file_path="test/path.pdf"
        )
        
        self.assertEqual(result, "doc123")
        self.mock_utils.upload.assert_called_with("test_sso", "test/path.pdf")


if __name__ == '__main__':
    unittest.main()